In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
%pip install patool
import patoolib
patoolib.extract_archive("/content/drive/My Drive/Adience/datasetAge.rar", outdir="/content")

     |████████████████████████████████| 81kB 2.2MB/s 
patool: Extracting /content/drive/My Drive/Adience/datasetAge.rar ...
patool: running /usr/bin/unrar x -- "/content/drive/My Drive/Adience/datasetAge.rar"
patool:     with cwd='/content'
patool: ... /content/drive/My Drive/Adience/datasetAge.rar extracted to `/content'.


'/content'

In [ ]:
import keras
import  tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, AveragePooling2D, PReLU, ReLU, Input, GlobalAveragePooling2D
from keras import optimizers, regularizers
from keras.preprocessing.image import ImageDataGenerator
import numpy as np
np.random.seed(48)
from keras.applications import vgg19
from keras.models import Model, load_model
from keras.optimizers import Adam
# input_shape = (224, 224, 3)
# vgg = vgg19.VGG19(include_top=False ,weights='imagenet', 
#                                      input_shape=input_shape)   
from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping, ReduceLROnPlateau

Using TensorFlow backend.


In [ ]:
class FixedImageDataGenerator(ImageDataGenerator):
  def standardize(self, x):
    if self.featurewise_center:
      img = tf.keras.applications.resnet50.preprocess_input(x)
      # x = ((x/255.) - 0.5) * 2.
    return x

In [ ]:
IMG_WIDTH = 256
IMG_HEIGHT = 256
CHANNELS = 3

In [ ]:
train_datagen = ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,)

train_datagen = FixedImageDataGenerator(train_datagen)
train_generator = train_datagen.flow_from_directory(
        '/content/datasetAge/train',
        target_size=(IMG_WIDTH, IMG_HEIGHT),
        batch_size=32,
        class_mode='categorical')

validate_datagen = ImageDataGenerator(
        featurewise_center=True,
        featurewise_std_normalization=True)

validate_generator = validate_datagen.flow_from_directory(
        '/content/datasetAge/test',
        target_size=(IMG_WIDTH, IMG_HEIGHT),
        batch_size=32,
        class_mode='categorical')

Found 13840 images belonging to 8 classes.
Found 3676 images belonging to 8 classes.


In [ ]:
!pip install efficientnet
import efficientnet.keras as efn 

In [ ]:
def efficientnetB0():
    model_input = Input(shape=(IMG_WIDTH, IMG_HEIGHT, CHANNELS))
    baseModel = efn.EfficientNetB0(weights='imagenet', include_top=False, input_tensor=model_input)
    # for layer in baseModel.layers:
	#     layer.trainable = False
    # for i, layer in enumerate(baseModel.layers):
    #     if "batch_normalization" in layer.name:
    #         baseModel.layers[i] = GroupNormalization(groups=32, axis=-1, epsilon=0.00001)
    X = baseModel.output
    X = GlobalAveragePooling2D()(X)
    X = Dense(4096, activation="relu", kernel_regularizer=regularizers.l1_l2(l1=0.1, l2=0.1))(X)
    X = Dropout(0.5)(X)
    X = Dense(8, activation="softmax")(X)
    model = Model(inputs=model_input, outputs=X)
    model.compile(optimizer=Adam(lr=1e-3, beta_1=0.9), loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [ ]:
efficientnetb0_model = efficientnetB0()

16809984/16804768 [==============================] - 2s 0us/step


In [ ]:
efficientnetb0_model.load_weights('/content/drive/My Drive/Adience/effnet-adience-age89.hdf5')

In [ ]:
filepath= "effnet-adience-age.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
early = EarlyStopping(monitor="val_accuracy", mode="max", patience=5)
rlr = ReduceLROnPlateau(monitor='val_loss', 
                        factor=0.5, 
                        patience=2, 
                        verbose=1, 
                        mode='auto', 
                        min_delta=0.0001)
callbacks_list = [checkpoint, early, rlr]

In [ ]:
batch_size = 32
epochs = 10
history = efficientnetb0_model.fit_generator(train_generator, 
                              steps_per_epoch = train_generator.samples // batch_size, 
                              epochs = epochs,
                              validation_data = validate_generator,
                              validation_steps= validate_generator.samples // batch_size, 
                              callbacks = callbacks_list,
                              verbose=1)

Epoch 1/10


/usr/local/lib/python3.6/dist-packages/keras_preprocessing/image/image_data_generator.py:720: UserWarning: This ImageDataGenerator specifies `featurewise_center`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '
/usr/local/lib/python3.6/dist-packages/keras_preprocessing/image/image_data_generator.py:728: UserWarning: This ImageDataGenerator specifies `featurewise_std_normalization`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '


432/432 [==============================] - 201s 465ms/step - loss: 8.8427 - accuracy: 0.9184 - val_loss: 8.7401 - val_accuracy: 0.8809

Epoch 00001: val_accuracy did not improve from 0.89431
Epoch 2/10
432/432 [==============================] - 201s 466ms/step - loss: 8.7533 - accuracy: 0.9183 - val_loss: 8.6176 - val_accuracy: 0.8747

Epoch 00002: val_accuracy did not improve from 0.89431
Epoch 3/10
432/432 [==============================] - 202s 468ms/step - loss: 8.6853 - accuracy: 0.9195 - val_loss: 8.7395 - val_accuracy: 0.8823

Epoch 00003: val_accuracy did not improve from 0.89431
Epoch 4/10
432/432 [==============================] - 202s 467ms/step - loss: 8.6369 - accuracy: 0.9202 - val_loss: 9.0194 - val_accuracy: 0.8854

Epoch 00004: val_accuracy did not improve from 0.89431

Epoch 00004: ReduceLROnPlateau reducing learning rate to 6.25000029685907e-05.
Epoch 5/10
432/432 [==============================] - 202s 467ms/step - loss: 4.6720 - accuracy: 0.9180 - val_loss: 4.6358 

In [ ]:
!cp '/content/effnet-adience-age89.hdf5' '/content/drive/My Drive/Adience'